In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os

from mpl_toolkits.axes_grid1 import make_axes_locatable

%matplotlib inline

In [ ]:
results_path = os.path.join(os.environ['HOME'], 'fits/neurocorr', 'exp2_values_pvc11_15_100_1000.npz')

In [ ]:
results = np.load(results_path)

In [ ]:
v_s_lfi = results['v_s_lfi']
v_s_sdkl = results['v_s_sdkl']
v_r_lfi = results['v_r_lfi']
v_r_sdkl = results['v_r_sdkl']
v_lfi = results['v_lfi'][:, :, 0]
v_sdkl = results['v_sdkl'][:, :, 0]

In [ ]:
dims = np.arange(v_s_lfi.shape[0]) + 2
n_dims = dims.size
n_dimlet_stim_combos = v_s_lfi.shape[1]
n_repeats = v_s_lfi.shape[2]

# Behavior of LFI/sDKL

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 4))

for idx in range(0, 1200, 4):
    axes[0].plot(dims, v_lfi[:, idx], color='gray', alpha=0.1)
    axes[1].plot(dims, v_sdkl[:, idx], color='gray', alpha=0.1)

    
axes[0].plot(dims, np.mean(v_lfi, axis=1), color='black', linewidth=2)
axes[1].plot(dims, np.mean(v_sdkl, axis=1), color='black', linewidth=2)

for ax in axes:
    ax.set_xlim([2, 15])
    ax.tick_params(labelsize=15)
    ax.set_xlabel(r'\textbf{Dimension}', fontsize=17)
    
axes[0].set_title(r'\textbf{LFI}', fontsize=20)
axes[1].set_title(r'\textbf{sD}\textsubscript{\textbf{KL}}', fontsize=20)

# Scatter Comparison at Different Dimensions

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20, 10))

for idx in range(n_repeats):
    axes[0].boxplot(
        [v_r_lfi[0, idx, :]],
        positions=[v_lfi[0, idx]],
        widths=0,
        boxprops={'linewidth' : 1.5, 'color': 'red', 'alpha': 0.1},
        medianprops={'marker' : '.', 'markersize' : 20, 'color' : 'red', 'alpha': 0.1},
        whiskerprops={'linewidth' : 1.5, 'color' : 'red', 'alpha': 0.1},
        showfliers=False
    )
    axes[0].boxplot(
        [v_s_lfi[0, idx, :]],
        positions=[v_lfi[0, idx]],
        widths=0,
        boxprops={'linewidth' : 1.5, 'color': 'gray', 'alpha': 0.2},
        medianprops={'marker' : '.', 'markersize' : 20, 'color' : 'gray', 'alpha': 0.2},
        whiskerprops={'linewidth' : 1.5, 'color' : 'gray', 'alpha': 0.2},
        showfliers=False
    )

    axes[1].boxplot(
        [v_r_lfi[8, idx, :]],
        positions=[v_lfi[8, idx]],
        widths=0,
        boxprops={'linewidth' : 1.5, 'color': 'red', 'alpha': 0.1},
        medianprops={'marker' : '.', 'markersize' : 20, 'color' : 'red', 'alpha': 0.1},
        whiskerprops={'linewidth' : 1.5, 'color' : 'red', 'alpha': 0.1},
        showfliers=False
    )
    axes[1].boxplot(
        [v_s_lfi[8, idx, :]],
        positions=[v_lfi[8, idx]],
        widths=0,
        boxprops={'linewidth' : 1.5, 'color': 'gray', 'alpha': 0.2},
        medianprops={'marker' : '.', 'markersize' : 20, 'color' : 'gray', 'alpha': 0.2},
        whiskerprops={'linewidth' : 1.5, 'color' : 'gray', 'alpha': 0.2},
        showfliers=False
    )

for ax in axes:
    ax.set_xlabel(r'\textbf{Observed LFI}')
    ax.set_ylabel(r'\textbf{LFI under Shuffle/Rotation}')
    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.set_xlim([1e-6, 10])
    ax.set_ylim([1e-6, 10])
    ax.plot(ax.get_xlim(), ax.get_ylim(), color='k')
axes[0].set_title(r'$D=2$', fontsize=25)
axes[1].set_title(r'$D=10$', fontsize=25)

# Cumulative of Histograms at Percentiles

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 5), sharex=True, sharey=True)

dim_idx = 0
percentiles = [2.5, 50, 97.5]

for percentile in percentiles:
    axes[0].hist(
        v_lfi[dim_idx] - np.percentile(v_s_lfi[dim_idx], percentile, -1),
        cumulative=True,
        density=True,
        bins=np.linspace(-0.01, 0.01, 51),
        histtype='step',
        label=f'{percentile}\%',
        linewidth=3)

    axes[1].hist(
        v_lfi[dim_idx] - np.percentile(v_r_lfi[dim_idx], percentile, -1),
        cumulative=True,
        density=True,
        bins=np.linspace(-0.01, 0.01, 51),
        histtype='step',
        linewidth=3)

for ax in axes:
    ax.axhline(0.5, linestyle='--', color='black')
    ax.axvline(0, linestyle='--', color='black')
    ax.tick_params(labelsize=15)
    ax.set_xticks([-0.01, 0, 0.01])
    ax.set_ylabel(r'\textbf{Cumulative Density}', fontsize=18)

axes[0].set_xlabel(r'\textbf{Difference in LFI (Shuffle)}', fontsize=18)
axes[1].set_xlabel(r'\textbf{Difference in LFI (Rotation)}', fontsize=18)
axes[0].legend(loc=2, prop={'size': 15})

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 5))

dim_idx = 5
percentiles = [2.5, 50, 97.5]
bins = np.linspace(-10, 10, 51)

for percentile in percentiles:
    axes[0].hist(
        v_sdkl[dim_idx] - np.percentile(v_s_sdkl[dim_idx], percentile, -1),
        cumulative=True,
        density=True,
        bins=bins,
        histtype='step',
        linewidth=3)

    axes[1].hist(
        v_sdkl[dim_idx] - np.percentile(v_r_sdkl[dim_idx], percentile, -1),
        cumulative=True,
        density=True,
        bins=bins,
        histtype='step',
        linewidth=3)

for ax in axes:
    ax.axhline(0.5, linestyle='--', color='black')
    ax.axvline(0, linestyle='--', color='black')

# Behavior of Zero-crossings across Dimensions

In [ ]:
percentiles = np.array([2.5, 25, 50, 75, 97.5])
n_percentiles = percentiles.size

dim_crossings_s = np.zeros((n_dims, n_percentiles))
dim_crossings_r = np.zeros((n_dims, n_percentiles))

for dim_idx, dim in enumerate(dims):
    for idx, percentile in enumerate(percentiles):
        dim_crossings_s[dim_idx, idx] = np.count_nonzero(
            (v_lfi[dim_idx] - np.percentile(v_s_lfi[dim_idx], percentile, -1)) < 0
        ) / n_dimlet_stim_combos
        dim_crossings_r[dim_idx, idx] = np.count_nonzero(
            (v_lfi[dim_idx] - np.percentile(v_r_lfi[dim_idx], percentile, -1)) < 0
        ) / n_dimlet_stim_combos

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 5), sharex=True, sharey=True)

for idx, percentile in enumerate(percentiles):
    axes[0].plot(dims, dim_crossings_s[:, idx], linewidth=2, label=f'{percentile}\%')
    axes[1].plot(dims, dim_crossings_r[:, idx], linewidth=2)
    
for ax in axes:
    ax.set_ylim([0, 1])
    ax.tick_params(labelsize=14)
    ax.set_xlim([2, 15])
    ax.set_ylim([0, 1.03])
    ax.set_xlabel(r'\textbf{Dimensionality}', fontsize=15)
    ax.set_ylabel(r'\textbf{Fraction of Dimlets with} $\Delta$\textbf{LFI}$<0$', fontsize=15)
    
axes[0].legend(loc=2, prop={'size': 12})

plt.tight_layout()

# Scatter across dimensions

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20, 10))

for idx in range(n_repeats):
    axes[0].boxplot(
        [v_r_sdkl[0, idx, :]],
        positions=[v_sdkl[0, idx]],
        widths=0,
        boxprops={'linewidth' : 1.5, 'color': 'red', 'alpha': 0.1},
        medianprops={'marker' : '.', 'markersize' : 20, 'color' : 'red', 'alpha': 0.1},
        whiskerprops={'linewidth' : 1.5, 'color' : 'red', 'alpha': 0.1},
        showfliers=False
    )
    axes[0].boxplot(
        [v_s_sdkl[0, idx, :]],
        positions=[v_sdkl[0, idx]],
        widths=0,
        boxprops={'linewidth' : 1.5, 'color': 'gray', 'alpha': 0.2},
        medianprops={'marker' : '.', 'markersize' : 20, 'color' : 'gray', 'alpha': 0.2},
        whiskerprops={'linewidth' : 1.5, 'color' : 'gray', 'alpha': 0.2},
        showfliers=False
    )

    axes[1].boxplot(
        [v_r_sdkl[8, idx, :]],
        positions=[v_sdkl[8, idx]],
        widths=0,
        boxprops={'linewidth' : 1.5, 'color': 'red', 'alpha': 0.1},
        medianprops={'marker' : '.', 'markersize' : 20, 'color' : 'red', 'alpha': 0.1},
        whiskerprops={'linewidth' : 1.5, 'color' : 'red', 'alpha': 0.1},
        showfliers=False
    )
    axes[1].boxplot(
        [v_s_sdkl[8, idx, :]],
        positions=[v_sdkl[8, idx]],
        widths=0,
        boxprops={'linewidth' : 1.5, 'color': 'gray', 'alpha': 0.2},
        medianprops={'marker' : '.', 'markersize' : 20, 'color' : 'gray', 'alpha': 0.2},
        whiskerprops={'linewidth' : 1.5, 'color' : 'gray', 'alpha': 0.2},
        showfliers=False
    )

for ax in axes:
    ax.set_xlabel(r'\textbf{Observed sDKL}')
    ax.set_ylabel(r'\textbf{sDKL under Shuffle/Rotation}')
    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.set_xlim([1e-2, 1e4])
    ax.set_ylim([1e-2, 1e4])
    ax.plot(ax.get_xlim(), ax.get_ylim(), color='k')
axes[0].set_title(r'$D=2$', fontsize=25)
axes[1].set_title(r'$D=10$', fontsize=25)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20, 10))

for idx, dim_idx in enumerate(range(dims.size)):
    color_s = plt.get_cmap('plasma')(int(idx / (dims.size - 1) * 255))
    color_r = plt.get_cmap('viridis_r')(int(idx / (dims.size - 1) * 255))
    axes[0].scatter(
        v_lfi[dim_idx],
        np.percentile(v_s_lfi[dim_idx], q=50, axis=1),
        s=25,
        color=color_s,
        alpha=0.1
    )
    axes[0].scatter(
        v_lfi[dim_idx],
        np.percentile(v_r_lfi[dim_idx], q=50, axis=1),
        s=25,
        color=color_r,
        alpha=0.1
    )
    axes[1].scatter(
        v_sdkl[dim_idx],
        np.percentile(v_s_sdkl[dim_idx], q=50, axis=1),
        s=25,
        color=color_s,
        alpha=0.1
    )
    axes[1].scatter(
        v_sdkl[dim_idx],
        np.percentile(v_r_sdkl[dim_idx], q=50, axis=1),
        s=25,
        color=color_r,
        alpha=0.1
    )

# colorbars
divider = make_axes_locatable(axes[1])
cax1 = divider.append_axes("right", size="5%", pad=0.2)
cb = fig.colorbar(
    mpl.cm.ScalarMappable(
        norm=mpl.colors.Normalize(vmin=2, vmax=15),
        cmap="viridis_r"
    ),
    cax=cax1)

cax2 = divider.append_axes("right", size="5%", pad=1)
cb2 = fig.colorbar(
    mpl.cm.ScalarMappable(
        norm=mpl.colors.Normalize(vmin=2, vmax=15),
        cmap="plasma"
    ),
    cax=cax2
)

# labels
axes[0].set_xlabel(r'\textbf{Observed LFI}')
axes[0].set_ylabel(r'\textbf{Median LFI under Shuffle/Rotation}')
axes[1].set_xlabel(r'\textbf{Observed sD}\textsubscript{\textbf{KL}}')
axes[1].set_ylabel(r'\textbf{Median sD}\textsubscript{\textbf{KL}}\textbf{ under Shuffle/Rotation}')

axes[0].set_xlim([1e-5, 1])
axes[0].set_ylim([1e-5, 1])
axes[1].set_xlim([1e-2, 1e3])
axes[1].set_ylim([1e-2, 1e3])

cb.ax.set_ylabel(r'\textbf{Dimension for Rotation Null Model}', labelpad=17, fontsize=20, rotation=270)
cb2.ax.set_ylabel(r'\textbf{Dimension for Shuffle Null Model}', labelpad=17, fontsize=20, rotation=270)

for ax in axes:
    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.plot(ax.get_xlim(), ax.get_ylim(), color='k')
    
plt.tight_layout()